In [3]:
import Pkg

In [4]:
Pkg.add("StructuralCausalModels")
Pkg.add("DataFrames")
Pkg.add("GLM")
Pkg.add("StatsModels")
Pkg.add("Plots")
Pkg.add("StatsPlots") 
Pkg.add("Printf")
Pkg.add("Random")
Pkg.add("CSV")


   Resolving package versions...
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\johnb\.julia\environments\v1.11\Project.toml`
  No 

In [5]:
# Cargar los paquetes para la sesión
using StructuralCausalModels, DataFrames, GLM, StatsModels, Plots, StatsPlots, Printf, Random, CSV

In [6]:
## 1.1 DAG y Simulación
# Definimos el grafo donde U1 y U2 son no observados
dag_original_str = "dag {
    U1 [unobserved]; U2 [unobserved]
    U1 -> X; U1 -> Z
    U2 -> Y; U2 -> Z
    X -> Y;  Z -> Y
}"
dag_original = dag""
draw(dag_original) 



LoadError: LoadError: UndefVarError: `@dag_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at In[6]:9

In [ ]:
# Simulación de datos para el DAG original
Random.seed!(111)
n_samples = 100000
true_effect_original = 1.0

U1 = randn(n_samples)
U2 = randn(n_samples)
Z  = 0.6 * U1 + 0.5 * U2 .+ randn(n_samples)
X  = 0.8 * U1 .+ randn(n_samples)
Y  = true_effect_original .* X + 0.4 .* Z + 0.9 .* U2 .+ randn(n_samples)

df_original = DataFrame(X=X, Y=Y, Z=Z)

In [ ]:
## 1.2 Regresiones con y sin control por Z
# Usamos `Y ~ 0 + ...` para un modelo sin intercepto
model_sin_Z = lm(@formula(Y ~ 0 + X), df_original)
model_con_Z = lm(@formula(Y ~ 0 + X + Z), df_original)

println("\n--- Regresión Y vs. X (Sin Control) ---")
println(model_sin_Z)
println("\n--- Regresión Y vs. X (Controlando por Z) ---")
println(model_con_Z)


In [ ]:
## 1.3 Gráfico de Coeficientes vs. Efecto Verdadero
# Extraer coeficientes e intervalos de confianza del 93%
coef_sin_Z = coef(model_sin_Z)[1]
conf_sin_Z = confint(model_sin_Z, 0.93)[1, :]

coef_con_Z = coef(model_con_Z)[1]
conf_con_Z = confint(model_con_Z, 0.93)[1, :]

plot_data = DataFrame(
    Model = ["Y vs. X", "Y vs. X, Z"],
    estimate = [coef_sin_Z, coef_con_Z],
    conf_low = [conf_sin_Z[1], conf_con_Z[1]],
    conf_high = [conf_sin_Z[2], conf_con_Z[2]]
)

# Crear el gráfico
@df plot_data bar(
    :Model, :estimate, yerror=(:estimate .- :conf_low, :conf_high .- :estimate),
    title="Estimación del Efecto de X en Y (Original)",
    ylabel="Coeficiente Estimado de X",
    legend=false,
    fill=["skyblue", "salmon"],
    alpha=0.8
)
hline!([true_effect_original], linestyle=:dash, color=:red, label="Efecto Verdadero")

# Guardar el gráfico
if !isdir("output"); mkdir("output"); end
savefig("output/part3_coefficients_plot_Julia.png")
println("\nGráfico exportado a 'output/part3_coefficients_plot_Julia.png'\n"

In [ ]:
# --- 2. DAG Modificado con Controles Observables ---

println("\n\n--- PARTE 3.2: DAG Modificado y Búsqueda de Controles ---")

## 2.1 Grafo y Simulación del DAG Modificado
dag_modificado_str = "dag{ U1->X; U1->Z; U2->Y; U2->Z; X->Y; Z->Y; Z->X }"
dag_modificado = dag""
# draw(dag_modificado) # Descomentar si tienes Graphviz

# Simulación donde U1 y U2 son observables
Random.seed!(456)
true_effect_modificado = 1.0

U1 = randn(n_samples)
U2 = randn(n_samples)
Z  = 1 .* U1 .+ 1 .* U2 .+ randn(n_samples)
X  = 1 .* U1 .+ 1 .* Z .+ randn(n_samples)
Y  = true_effect_modificado .* X .+ 1 .* Z .+ 1 .* U2 .+ randn(n_samples)

df_modified = DataFrame(X=X, Y=Y, Z=Z, U1=U1, U2=U2)

In [ ]:
## 2.2 Correr las 8 Regresiones
formulas = [
    "None"      => @formula(Y ~ 0 + X),
    "Z"         => @formula(Y ~ 0 + X + Z),
    "U1"        => @formula(Y ~ 0 + X + U1),
    "U2"        => @formula(Y ~ 0 + X + U2),
    "Z, U1"     => @formula(Y ~ 0 + X + Z + U1),
    "Z, U2"     => @formula(Y ~ 0 + X + Z + U2),
    "U1, U2"    => @formula(Y ~ 0 + X + U1 + U2),
    "Z, U1, U2" => @formula(Y ~ 0 + X + Z + U1 + U2)
]

results_list = []
for (controls, formula) in formulas
    model = lm(formula, df_modified)
    ct = coeftable(model)
    # Encontrar la fila correspondiente a 'X'
    x_row = findfirst(x -> x == "X", ct.rownms)
    beta = ct.cols[1][x_row]
    se = ct.cols[2][x_row]
    push!(results_list, (Controls=controls, β=beta, SE=se))
end
results_df = DataFrame(results_list)


In [ ]:
## 2.3 Presentar y Exportar la Tabla de Resultados
println("\n--- Tabla de Resultados: Escenario Modificado ---")
@printf "El efecto causal verdadero es: %.1f\n\n" true_effect_modificado
println(results_df)
CSV.write("output/regression_results_Julia.txt", results_df, delim='\t')
println("\n✅ Tabla exportada a 'output/regression_results_Julia.txt'\n")

# --- 3. Análisis Final y Conclusiones ---


In [3]:
## 3.1 ¿Qué controles funcionan?
println("--- 3.1 ¿Qué controles funcionan? ---")
tolerance = 0.02
good_controls_df = filter(row -> abs(row.β - true_effect_modificado) < tolerance, results_df)
@printf "Modelos que estiman el efecto con una tolerancia de +/- %.2f:\n" tolerance
println(good_controls_df)

--- 3.1 ¿Qué controles funcionan? ---


LoadError: UndefVarError: `results_df` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
## 3.2 ¿Cuál es el conjunto mínimo suficiente?
println("\n--- 3.2 ¿Cuál es el conjunto mínimo suficiente? ---")
good_sets = [Set(split(c, ", ")) for c in good_controls_df.Controls]

minimal_sufficient_sets = []
for current_set in good_sets
    is_minimal = true
    for other_set in good_sets
        if current_set != other_set && issubset(other_set, current_set)
            is_minimal = false
            break
        end
    end
    if is_minimal
        push!(minimal_sufficient_sets, current_set)
    end
end

println("El/Los conjunto(s) mínimo(s) suficiente(s) es/son:")
println(minimal_sufficient_sets)

In [2]:
## 3.3 Intuición Causal
println("""
--- 3.3 Intuición Causal del Resultado ---

La intuición de por qué los conjuntos de controles {Z, U1} y {Z, U2} proporcionan una estimación correcta se basa en su capacidad para satisfacer el criterio de la puerta trasera ('back-door criterion'). Este criterio exige que cerremos todos los caminos no causales que generan confusión entre el tratamiento X y el resultado Y.

En el grafo causal modificado, existen tres caminos de puerta trasera que conectan a X con Y: (1) X <- Z -> Y, (2) X <- U1 -> Z -> Y, y (3) X <- Z <- U2 -> Y.

Al utilizar el conjunto de controles {Z, U1}, logramos bloquear estos tres caminos. Sin embargo, surge una complicación: el nodo Z es un colisionador en la ruta U1 -> Z <- U2. Al controlar por Z, abrimos este camino, creando un nuevo camino de puerta trasera: X <- U1 -- U2 -> Y.

Aquí radica la intuición clave: aunque controlar por Z abre este nuevo camino espurio, el conjunto {Z, U1} también incluye el control sobre U1. Este control adicional sobre U1 cierra inmediatamente el nuevo camino que acababa de abrirse. Por lo tanto, el conjunto {Z, U1} tiene éxito porque no solo cierra todos los caminos de puerta trasera originales, sino que también neutraliza el nuevo camino de sesgo que él mismo crea. La misma lógica se aplica de forma simétrica para el conjunto {Z, U2}.
""")

--- 3.3 Intuición Causal del Resultado ---

La intuición de por qué los conjuntos de controles {Z, U1} y {Z, U2} proporcionan una estimación correcta se basa en su capacidad para satisfacer el criterio de la puerta trasera ('back-door criterion'). Este criterio exige que cerremos todos los caminos no causales que generan confusión entre el tratamiento X y el resultado Y.

En el grafo causal modificado, existen tres caminos de puerta trasera que conectan a X con Y: (1) X <- Z -> Y, (2) X <- U1 -> Z -> Y, y (3) X <- Z <- U2 -> Y.

Al utilizar el conjunto de controles {Z, U1}, logramos bloquear estos tres caminos. Sin embargo, surge una complicación: el nodo Z es un colisionador en la ruta U1 -> Z <- U2. Al controlar por Z, abrimos este camino, creando un nuevo camino de puerta trasera: X <- U1 -- U2 -> Y.

Aquí radica la intuición clave: aunque controlar por Z abre este nuevo camino espurio, el conjunto {Z, U1} también incluye el control sobre U1. Este control adicional sobre U1 cierra i